## Robot Run

In a certain terrain a Robot (R) has to navigate to a Goal (G) avoiding Blocks (B) and human player (H)

In [1]:
terrain = [
    ["R", "_", "_", "_"],
    ["_", "_", "B", "_"],
    ["H", "_", "B", "_"],
    ["B", "_", "G", "_"]   
]

In [26]:
from copy import deepcopy

robot_symbol = 'R'
robot_win_symbol = '*'
goal_symbol = 'G'
human_symbol = 'H'
blank_symbol = '_'

def is_robot_win(state):
    for row in state:
        for field in row:
            if field == robot_win_symbol:
                return True
    return False   

def as_string(state):
    s = ''
    for row in state:
        row_string = ''
        for field in row:
            row_string += field + ' '
        s += row_string + '\n'
    return s

def locate_robot(state):
    for row_index, row in enumerate(state):
        for column_index, field in enumerate(row):
            if field == robot_symbol:
                return (row_index, column_index)

def check_robot(state, robot):
    max_row = len(state) - 1
    max_column = len(state[0]) - 1
    if robot[0] < 0 or robot[0] > max_row or robot[1] < 0 or robot[1] > max_column:
        return False
    symbol = state[robot[0]][robot[1]]
    if symbol != blank_symbol and symbol != goal_symbol:
        return False
    return True
            
def robot_moves(state):
    robot = locate_robot(state)
    left = (robot[0], robot[1] - 1)
    right = (robot[0], robot[1] + 1)
    up = (robot[0] - 1, robot[1])
    down = (robot[0] + 1, robot[1])
    valid_moves = [move for move in (left, right, down, up) if check_robot(state, move)]
    return valid_moves
            
def place_robot(state, robot):
    old_robot = locate_robot(state)
    new_state = deepcopy(state)
    new_state[old_robot[0]][old_robot[1]] = blank_symbol
    if new_state[robot[0]][robot[1]] == goal_symbol:
        new_state[robot[0]][robot[1]] = robot_win_symbol
    else:
        new_state[robot[0]][robot[1]] = robot_symbol
    return new_state

def expand(state):
    valid_moves = robot_moves(state)
    new_states = [(robot, place_robot(state, robot)) for robot in valid_moves]
    return new_states

In [88]:
# https://en.wikipedia.org/wiki/Depth-first_search
# 1  procedure DFS(G,v):
# 2      label v as discovered
# 3      for all edges from v to w in G.adjacentEdges(v) do
# 4          if vertex w is not labeled as discovered then
# 5              recursively call DFS(G,w)

def depth_first_search(state, max_depth=10, debug=False, closed_list=[], depth = 0, path=[]):
    if as_string(state) in closed_list or depth > max_depth:
        return None
    
    if debug:
        print('depth', depth)
        print('closed_list', closed_list)
        print('path', path)
        print('state', as_string(state))
        
    if is_robot_win(state):
        return path
    
    closed_list = closed_list + [as_string(state)]
    
    for move, next_state in expand(state):
        new_path = path + [move]
        res = depth_first_search(next_state, max_depth, debug, closed_list, depth + 1, new_path)
        if res:
            return res

In [89]:
# this does not find the best path
depth_first_search(terrain, 7)

[(0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (3, 2)]

In [90]:
# if we restrict ourselves to only five levels we find the best
depth_first_search(terrain, 5)

[(0, 1), (1, 1), (2, 1), (3, 1), (3, 2)]

## Understanding the search in a simplified terrain

In [91]:
simple_terrain = [
    ["R", "_", "_", "_"],
    ["B", "_", "G", "_"] 
]

In [92]:
depth_first_search(simple_terrain, debug=True)

depth 0
closed_list []
path []
state R _ _ _ 
B _ G _ 

depth 1
closed_list ['R _ _ _ \nB _ G _ \n']
path [(0, 1)]
state _ R _ _ 
B _ G _ 

depth 2
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n']
path [(0, 1), (0, 2)]
state _ _ R _ 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3)]
state _ _ _ R 
B _ G _ 

depth 4
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n', '_ _ _ R \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3), (1, 3)]
state _ _ _ _ 
B _ G R 

depth 5
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n', '_ _ _ R \nB _ G _ \n', '_ _ _ _ \nB _ G R \n']
path [(0, 1), (0, 2), (0, 3), (1, 3), (1, 2)]
state _ _ _ _ 
B _ * _ 



[(0, 1), (0, 2), (0, 3), (1, 3), (1, 2)]

In [93]:
depth_first_search(simple_terrain, max_depth=3, debug=True)

depth 0
closed_list []
path []
state R _ _ _ 
B _ G _ 

depth 1
closed_list ['R _ _ _ \nB _ G _ \n']
path [(0, 1)]
state _ R _ _ 
B _ G _ 

depth 2
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n']
path [(0, 1), (0, 2)]
state _ _ R _ 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3)]
state _ _ _ R 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (1, 2)]
state _ _ _ _ 
B _ * _ 



[(0, 1), (0, 2), (1, 2)]

## Can we find the best path without restricting the depth?